# Data Wrangling: Join, Combine, and Reshape

## Hierarchichal Indexing

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.Series(np.random.randn(9),index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.297477
   2   -0.590742
   3   -0.279464
b  1    1.638806
   3    0.428699
c  1   -0.879176
   2   -1.133598
d  2    0.278122
   3    0.466430
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
data["b"]

1    1.638806
3    0.428699
dtype: float64

In [6]:
data["b":"c"]

b  1    1.638806
   3    0.428699
c  1   -0.879176
   2   -1.133598
dtype: float64

In [7]:
data.loc[["b", "d"]]

b  1    1.638806
   3    0.428699
d  2    0.278122
   3    0.466430
dtype: float64

Selection is even possible from an "inner" level

In [8]:
data.loc[:, 2]

a   -0.590742
c   -1.133598
d    0.278122
dtype: float64

In [9]:
data.unstack()

,1,2,3
a,-0.297477,-0.590742,-0.279464
b,1.638806,NaN,0.428699
c,-0.879176,-1.133598,NaN
d,NaN,0.278122,0.466430


In [10]:
data.unstack().stack()

a  1   -0.297477
   2   -0.590742
   3   -0.279464
b  1    1.638806
   3    0.428699
c  1   -0.879176
   2   -1.133598
d  2    0.278122
   3    0.466430
dtype: float64

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ["key1", "key2"]

frame.columns.names = ["state", "color"]

In [14]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame["Ohio"]

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

### Reordering and Sorting Levels

**swaplevel** takes two level numbers or names and returns a new object with the levels interchanged

In [15]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

**sort_index** sorts the data using only the values in a single level

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statics by Level

In [18]:
frame.sum(level="key2")

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.sum(level="color", axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame`s columns

In [21]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


**set_index** function will create a new DataFrame using one or more of its columns

In [22]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets